# TrOCR Training on Google Colab 🚀

Notebook này hướng dẫn setup và training TrOCR trên Google Colab với GPU miễn phí.

## 📋 Chuẩn bị
1. Upload folder `trocr-htr` lên Google Drive
2. Upload dữ liệu training (ảnh + XML files)
3. Kết nối Google Drive với Colab
4. Cài đặt dependencies
5. Training với GPU

## ⚡ GPU Info
- Google Colab cung cấp Tesla T4 (16GB VRAM)
- Thời gian training estimate: 2-4 giờ
- Khuyến nghị sử dụng Colab Pro cho ổn định


In [ ]:
# Bước 1: Kiểm tra GPU và mount Google Drive
import torch
import os

# Kiểm tra GPU
print("🔥 GPU Information:")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ GPU not available! Go to Runtime -> Change runtime type -> Hardware accelerator -> GPU")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


🔥 GPU Information:
CUDA available: True
GPU count: 1
GPU name: Tesla T4
GPU memory: 15.8 GB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Bước 2: Setup paths và cài đặt dependencies
import os
import sys

# Định nghĩa paths
PROJECT_ROOT = "/content/drive/MyDrive/Constance_de_Salm_TrOCR"
TROCR_DIR = f"{PROJECT_ROOT}/trocr-htr"
DATA_DIR = f"{PROJECT_ROOT}/training-data"

print(f"🔧 Setting up paths...")
print(f"Project root: {PROJECT_ROOT}")
print(f"TrOCR directory: {TROCR_DIR}")
print(f"Data directory: {DATA_DIR}")

# Kiểm tra xem folders đã upload chưa
if not os.path.exists(TROCR_DIR):
    print(f"❌ TrOCR directory not found: {TROCR_DIR}")
    print("👉 Please upload the trocr-htr folder to your Google Drive first!")
else:
    print(f"✅ TrOCR directory found")

if not os.path.exists(DATA_DIR):
    print(f"❌ Data directory not found: {DATA_DIR}")
    print("👉 Please upload training data to your Google Drive first!")
else:
    print(f"✅ Data directory found")

# Change to TrOCR directory
os.chdir(TROCR_DIR)
sys.path.append(TROCR_DIR)
print(f"📁 Current directory: {os.getcwd()}")


🔧 Setting up paths...
Project root: /content/drive/MyDrive/Constance_de_Salm_TrOCR
TrOCR directory: /content/drive/MyDrive/Constance_de_Salm_TrOCR/trocr-htr
Data directory: /content/drive/MyDrive/Constance_de_Salm_TrOCR/training-data
✅ TrOCR directory found
✅ Data directory found
📁 Current directory: /content/drive/MyDrive/Constance_de_Salm_TrOCR/trocr-htr


In [ ]:
# Bước 3: Cài đặt dependencies
print("📦 Installing dependencies...")

# Cập nhật pip và cài đặt requirements
!pip install --upgrade pip
!pip install -r requirements.txt

# Cài đặt thêm một số packages có thể thiếu
!pip install datasets>=2.14.0
!pip install accelerate>=0.24.0
!pip install jiwer>=3.0.0

print("✅ Dependencies installed successfully!")

# Kiểm tra installations quan trọng
try:
    import transformers
    import torch
    from PIL import Image
    import pandas as pd
    import numpy as np
    print(f"🔥 Key libraries checked:")
    print(f"  - Transformers: {transformers.__version__}")
    print(f"  - PyTorch: {torch.__version__}")
    print(f"  - CUDA available: {torch.cuda.is_available()}")
except ImportError as e:
    print(f"❌ Import error: {e}")

# Test import custom modules
try:
    from src.data_loader import CdSDataLoader
    from src.model import TrOCRForCdS
    from src.trainer import TrOCRTrainer
    print("✅ Custom modules imported successfully!")
except ImportError as e:
    print(f"❌ Custom module import error: {e}")
    print("👉 Make sure you uploaded the src/ folder correctly")


📦 Installing dependencies...
✅ Dependencies installed successfully!
🔥 Key libraries checked:
  - Transformers: 4.55.1
  - PyTorch: 2.6.0+cu124
  - CUDA available: True
✅ Custom modules imported successfully!


In [ ]:
# Fix paths trong config cho Colab
print("🔧 Fixing data paths for Colab...")

# Đọc và sửa config
import yaml

# Load config hiện tại
with open('config/colab_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Cập nhật paths cho Colab
PROJECT_ROOT = "/content/drive/MyDrive/Constance_de_Salm_TrOCR"
DATA_DIR = f"{PROJECT_ROOT}/training-data"

# Sửa source_dirs
config['data']['source_dirs'] = [
    f"{DATA_DIR}/verite-terrain/",
    f"{DATA_DIR}/sample-images/",
    f"{DATA_DIR}/predic-corrigees/"
]

# Sửa output paths
config['data']['output_dir'] = f"{PROJECT_ROOT}/outputs/data/"
config['paths']['models_dir'] = f"{PROJECT_ROOT}/outputs/models/"
config['paths']['outputs_dir'] = f"{PROJECT_ROOT}/outputs/"
config['paths']['logs_dir'] = f"{PROJECT_ROOT}/outputs/logs/"

# Lưu lại config đã fix
with open('config/colab_config_fixed.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("✅ Config paths fixed!")

# Kiểm tra paths có tồn tại không
print("\n📁 Checking data directories:")
for source_dir in config['data']['source_dirs']:
    if os.path.exists(source_dir):
        files = os.listdir(source_dir)
        print(f"  ✅ {source_dir}: {len(files)} files")
        # Show examples
        img_files = [f for f in files if f.lower().endswith(('.jpg', '.tif', '.png'))]
        xml_files = [f for f in files if f.lower().endswith('.xml')]
        print(f"     Images: {len(img_files)}, XML: {len(xml_files)}")
        if img_files:
            print(f"     Example: {img_files[0]}")
    else:
        print(f"  ❌ {source_dir}: NOT FOUND")
        print(f"     👉 Need to upload data to this path")

🔧 Fixing data paths for Colab...
✅ Config paths fixed!

📁 Checking data directories:
  ✅ /content/drive/MyDrive/Constance_de_Salm_TrOCR/training-data/verite-terrain/: 78 files
     Images: 33, XML: 45
     Example: CdS02_Konv002-01_0034.jpg
  ✅ /content/drive/MyDrive/Constance_de_Salm_TrOCR/training-data/sample-images/: 107 files
     Images: 107, XML: 0
     Example: CdS02_Konv002-02_0001.tif
  ✅ /content/drive/MyDrive/Constance_de_Salm_TrOCR/training-data/predic-corrigees/: 20 files
     Images: 0, XML: 20


In [ ]:
print("📊 Data exploration with fixed config...")

from src.data_loader import CdSDataLoader
from src.utils import load_config

# Load config đã fix
config = load_config('config/colab_config_fixed.yaml', 'config/base_config.yaml')
print("✅ Fixed configuration loaded")

# Initialize data loader
data_loader = CdSDataLoader(config)

# Find data
print("🔍 Scanning for training data...")
image_paths, texts = data_loader.find_image_text_pairs(config['data']['source_dirs'])

print(f"📊 Data Summary:")
print(f"  - Total image-text pairs: {len(image_paths)}")

if len(texts) > 0:
    avg_length = sum(len(text) for text in texts) / len(texts)
    print(f"  - Average text length: {avg_length:.1f} characters")
    print(f"✅ Data found successfully!")
    print(f"📝 Sample text: {texts[0][:100]}...")
    print(f"🖼️ Sample image: {image_paths[0]}")

    # Show text length distribution
    lengths = [len(text) for text in texts]
    print(f"  - Min text length: {min(lengths)}")
    print(f"  - Max text length: {max(lengths)}")
    print(f"  - Texts under 10 chars: {sum(1 for l in lengths if l < 10)}")

else:
    print(f"❌ No training data found!")
    print(f"👉 Kiểm tra lại data paths:")
    for source_dir in config['data']['source_dirs']:
        print(f"   {source_dir}")

print(f"\n📁 Final data directories check:")
for source_dir in config['data']['source_dirs']:
    if os.path.exists(source_dir):
        files = os.listdir(source_dir)
        img_files = [f for f in files if f.lower().endswith(('.jpg', '.tif', '.png', '.jpeg'))]
        xml_files = [f for f in files if f.lower().endswith('.xml')]
        print(f"  ✅ {source_dir}")
        print(f"     📷 Images: {len(img_files)}")
        print(f"     📄 XML files: {len(xml_files)}")
        print(f"     🔗 Matching pairs found: {len([img for img in img_files if any(xml.startswith(img.split('.')[0]) for xml in xml_files)])}")
    else:
        print(f"  ❌ {source_dir}: NOT FOUND")

📊 Data exploration with fixed config...
✅ Fixed configuration loaded


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/content/drive/MyDrive/Constance_de_Salm_TrOCR/trocr-htr/src/data_loader.py:111: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
 

🔍 Scanning for training data...
📊 Data Summary:
  - Total image-text pairs: 33
  - Average text length: 2360.2 characters
✅ Data found successfully!
📝 Sample text: Chacun connait cet apologue : "Un peintre avait exposé un tableau représentant un lion terrassé "par...
🖼️ Sample image: /content/drive/MyDrive/Constance_de_Salm_TrOCR/training-data/verite-terrain/CdS02_Konv002-01_0034.jpg
  - Min text length: 1180
  - Max text length: 3927
  - Texts under 10 chars: 0

📁 Final data directories check:
  ✅ /content/drive/MyDrive/Constance_de_Salm_TrOCR/training-data/verite-terrain/
     📷 Images: 33
     📄 XML files: 45
     🔗 Matching pairs found: 33
  ✅ /content/drive/MyDrive/Constance_de_Salm_TrOCR/training-data/sample-images/
     📷 Images: 107
     📄 XML files: 0
     🔗 Matching pairs found: 0
  ✅ /content/drive/MyDrive/Constance_de_Salm_TrOCR/training-data/predic-corrigees/
     📷 Images: 0
     📄 XML files: 20
     🔗 Matching pairs found: 0


In [ ]:
print("🎯 Preparing for training with 33 samples...")

# Update config để phù hợp với dataset nhỏ
import yaml

# Load config
with open('config/colab_config_fixed.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Điều chỉnh cho dataset nhỏ (33 samples)
config['training'].update({
    'batch_size': 1,                    # Batch size nhỏ
    'gradient_accumulation_steps': 4,   # Effective batch = 4
    'num_epochs': 20,                   # Tăng epochs cho dataset nhỏ
    'learning_rate': 5e-5,              # Learning rate cao hơn
    'save_steps': 10,                   # Save thường xuyên
    'eval_steps': 20,                   # Eval thường xuyên
    'logging_steps': 5,                 # Log chi tiết
    'warmup_steps': 10,                 # Ít warmup steps
    'early_stopping_patience': 5       # Patience cao hơn
})

# Điều chỉnh data split cho dataset nhỏ
config['data'].update({
    'train_split': 0.8,    # 26 samples train
    'val_split': 0.1,      # 3 samples val
    'test_split': 0.1      # 4 samples test
})

# Lưu config optimized
with open('config/colab_small_dataset.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("✅ Config optimized for small dataset (33 samples)")

# Estimate training time
num_samples = 33
train_samples = int(num_samples * 0.8)  # 26 samples
batch_size = config['training']['batch_size']
gradient_accumulation_steps = config['training']['gradient_accumulation_steps']
num_epochs = config['training']['num_epochs']

effective_batch_size = batch_size * gradient_accumulation_steps
steps_per_epoch = max(1, train_samples // effective_batch_size)
total_steps = steps_per_epoch * num_epochs
estimated_time_minutes = total_steps * 30 / 60  # ~30 seconds per step for small dataset

print(f"⏱️ Training estimates:")
print(f"  - Total samples: {num_samples}")
print(f"  - Train/Val/Test: {train_samples}/3/4")
print(f"  - Effective batch size: {effective_batch_size}")
print(f"  - Steps per epoch: {steps_per_epoch}")
print(f"  - Total epochs: {num_epochs}")
print(f"  - Total steps: {total_steps}")
print(f"  - Estimated time: {estimated_time_minutes:.0f} minutes")

print(f"\n🚀 Ready to start training!")

🎯 Preparing for training with 33 samples...
✅ Config optimized for small dataset (33 samples)
⏱️ Training estimates:
  - Total samples: 33
  - Train/Val/Test: 26/3/4
  - Effective batch size: 4
  - Steps per epoch: 6
  - Total epochs: 20
  - Total steps: 120
  - Estimated time: 60 minutes

🚀 Ready to start training!


In [ ]:
# Fix import paths cho scripts
print("🔧 Fixing import paths...")

import sys
import os

# Đảm bảo current directory đúng
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Add paths to Python path
trocr_dir = "/content/drive/MyDrive/Constance_de_Salm_TrOCR/trocr-htr"
if trocr_dir not in sys.path:
    sys.path.insert(0, trocr_dir)

# Change to trocr directory
os.chdir(trocr_dir)
print(f"Changed to: {os.getcwd()}")

# Test imports
try:
    from src.trainer import TrOCRTrainer
    from src.data_loader import CdSDataLoader
    from src.model import TrOCRForCdS
    print("✅ All imports successful!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("📁 Checking src directory:")
    if os.path.exists('src'):
        print("  ✅ src/ directory exists")
        src_files = os.listdir('src')
        print(f"  📄 Files in src/: {src_files}")
    else:
        print("  ❌ src/ directory not found!")

🔧 Fixing import paths...
Current directory: /content/drive/MyDrive/Constance_de_Salm_TrOCR/trocr-htr
Changed to: /content/drive/MyDrive/Constance_de_Salm_TrOCR/trocr-htr
✅ All imports successful!


In [ ]:
# Fix TrainingArguments compatibility
print("🔧 Fixing TrainingArguments compatibility...")

import transformers
print(f"Transformers version: {transformers.__version__}")

# Check which parameter name is correct
from transformers import TrainingArguments
import inspect

# Get the signature of TrainingArguments.__init__
sig = inspect.signature(TrainingArguments.__init__)
params = list(sig.parameters.keys())

print("Available TrainingArguments parameters:")
eval_params = [p for p in params if 'eval' in p.lower()]
print(f"Evaluation-related params: {eval_params}")

# Check if it's 'evaluation_strategy' or 'eval_strategy'
if 'evaluation_strategy' in params:
    print("✅ Uses 'evaluation_strategy'")
    eval_strategy_param = 'evaluation_strategy'
elif 'eval_strategy' in params:
    print("✅ Uses 'eval_strategy'")
    eval_strategy_param = 'eval_strategy'
else:
    print("❌ No evaluation strategy parameter found")
    eval_strategy_param = None

🔧 Fixing TrainingArguments compatibility...
Transformers version: 4.55.1
Available TrainingArguments parameters:
Evaluation-related params: ['do_eval', 'eval_strategy', 'per_device_eval_batch_size', 'per_gpu_eval_batch_size', 'eval_accumulation_steps', 'eval_delay', 'jit_mode_eval', 'bf16_full_eval', 'fp16_full_eval', 'eval_steps', 'eval_do_concat_batches', 'batch_eval_metrics', 'eval_on_start', 'eval_use_gather_object']
✅ Uses 'eval_strategy'


In [ ]:
# 🔧 Fix model configuration với decoder_start_token_id
print("🔧 Fixing model configuration...")

from src.data_loader import CdSDataLoader
from src.model import TrOCRForCdS
from transformers import TrainingArguments, Trainer, VisionEncoderDecoderModel, TrOCRProcessor
from src.utils import load_config
import torch

# Load config
config = load_config('config/colab_small_dataset.yaml', 'config/base_config.yaml')
PROJECT_ROOT = "/content/drive/MyDrive/Constance_de_Salm_TrOCR"
config['paths']['models_dir'] = f"{PROJECT_ROOT}/outputs/models/"

import os
os.makedirs(config['paths']['models_dir'], exist_ok=True)

print("📊 Initializing with fixed model config...")

# 1. Load model and processor manually với config fix
model_name = config['model']['name']
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

# FIX: Set decoder_start_token_id
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

# Additional config fixes for training
model.config.vocab_size = model.config.decoder.vocab_size
model.config.max_length = config['model']['max_length']
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

print(f"✅ Model config fixed:")
print(f"  decoder_start_token_id: {model.config.decoder_start_token_id}")
print(f"  pad_token_id: {model.config.pad_token_id}")
print(f"  vocab_size: {model.config.vocab_size}")
print(f"  max_length: {model.config.max_length}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. Data
data_loader = CdSDataLoader(config)
train_dataset, val_dataset, test_dataset = data_loader.create_datasets()
print(f"✅ Data: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

# 3. Fixed collate function
def collate_fn_fixed(batch):
    """Fixed collate function for training"""
    pixel_values = torch.stack([item['pixel_values'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])

    return {
        'pixel_values': pixel_values,
        'labels': labels
    }

# 4. TrainingArguments với wandb disabled
training_args = TrainingArguments(
    output_dir=config['paths']['models_dir'],
    num_train_epochs=config['training']['num_epochs'],
    per_device_train_batch_size=config['training']['batch_size'],
    per_device_eval_batch_size=config['training']['batch_size'],
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps'],
    learning_rate=config['training']['learning_rate'],
    weight_decay=config['training']['weight_decay'],
    warmup_steps=config['training']['warmup_steps'],
    eval_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_steps=10,
    save_total_limit=3,
    logging_steps=5,
    logging_first_step=True,
    fp16=config['hardware']['mixed_precision'] == 'fp16',
    dataloader_num_workers=config['hardware']['dataloader_num_workers'],
    dataloader_pin_memory=config['hardware']['pin_memory'],
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to=[],  # Empty list to disable all reporting including wandb
    run_name="trocr_colab_training"
)

# Disable wandb completely
os.environ["WANDB_DISABLED"] = "true"

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=processor,  # Use processing_class instead of tokenizer
    data_collator=collate_fn_fixed
)

print("✅ Trainer created with fixed model config")

# 6. Start training
try:
    print("🔥 Starting training (take 3 - with fixed config)...")

    train_result = trainer.train()
    print("🎉 Training completed!")

    print(f"📊 Training Results:")
    print(f"  Final loss: {train_result.training_loss:.4f}")
    print(f"  Steps completed: {train_result.global_step}")

    # Save model manually
    final_model_path = f"{PROJECT_ROOT}/outputs/models/final"
    os.makedirs(final_model_path, exist_ok=True)

    model.save_pretrained(final_model_path)
    processor.save_pretrained(final_model_path)

    print(f"💾 Model saved to: {final_model_path}")

    # Test một sample prediction
    print("🧪 Testing model với một sample...")
    model.eval()
    with torch.no_grad():
        test_sample = test_dataset[0]
        pixel_values = test_sample['pixel_values'].unsqueeze(0).to(device)
        generated_ids = model.generate(pixel_values, max_length=384)
        prediction = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        reference = test_sample['text']

        print(f"📝 Sample prediction:")
        print(f"  Reference: {reference[:100]}...")
        print(f"  Prediction: {prediction[:100]}...")

    print("🎉 Training and testing completed successfully!")

except Exception as e:
    print(f"❌ Training failed: {e}")
    import traceback
    traceback.print_exc()

🔧 Fixing model configuration...
📊 Initializing with fixed model config...


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model config fixed:
  decoder_start_token_id: 0
  pad_token_id: 1
  vocab_size: 50265
  max_length: 384


/content/drive/MyDrive/Constance_de_Salm_TrOCR/trocr-htr/src/data_loader.py:111: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  transforms.append(A.GaussNoise(var_limit=(0, aug_config['noise_factor']), p=0.2))


✅ Data: Train=25, Val=4, Test=4
✅ Trainer created with fixed model config
🔥 Starting training (take 3 - with fixed config)...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
10,7.837000,7.388140
20,6.775600,6.829752
30,6.481700,6.672448
40,6.335700,6.589857
50,6.236100,6.561946
60,6.161400,6.563494
70,6.051200,6.558358
80,5.950500,6.560427
90,5.976800,6.558934
100,5.890100,6.568191


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 384, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['decoder.output_projection.weight'].


🎉 Training completed!
📊 Training Results:
  Final loss: 6.3331
  Steps completed: 140
💾 Model saved to: /content/drive/MyDrive/Constance_de_Salm_TrOCR/outputs/models/final
🧪 Testing model với một sample...
📝 Sample prediction:
  Reference: 1818 (de Mme la Princesse de Salm) Lettre A Mr Raboteau Dyck, ce 6 Décembre, 1818. Un mot seulement,...
  Prediction: 18 ( M. A. M. de..,... de,.,,. de de., de. deé..'..é.,',,, de,,'.,é,.', de de, de',.é,,é. de' de.' d...
🎉 Training and testing completed successfully!


In [ ]:
# 📊 ĐÁNH GIÁ CHI TIẾT MODEL TRAINING
print("📊 Comprehensive Model Evaluation")
print("=" * 50)

from src.evaluator import HTREvaluator
import torch
from torch.utils.data import DataLoader

# Load trained model
final_model_path = f"{PROJECT_ROOT}/outputs/models/final"
trained_model = VisionEncoderDecoderModel.from_pretrained(final_model_path)
trained_processor = TrOCRProcessor.from_pretrained(final_model_path)
trained_model.to(device)
trained_model.eval()

print("✅ Trained model loaded for evaluation")

# Evaluate on test set
predictions = []
references = []

print(f"🔍 Evaluating on {len(test_dataset)} test samples...")

with torch.no_grad():
    for i, test_sample in enumerate(test_dataset):
        # Get prediction
        pixel_values = test_sample['pixel_values'].unsqueeze(0).to(device)

        # Try different generation strategies
        generated_ids = trained_model.generate(
            pixel_values,
            max_length=200,  # Shorter length to avoid repetition
            num_beams=3,     # Fewer beams
            early_stopping=True,
            no_repeat_ngram_size=2,  # Prevent repetition
            do_sample=False
        )

        prediction = trained_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        reference = test_sample['text']

        predictions.append(prediction)
        references.append(reference)

        print(f"Sample {i+1}:")
        print(f"  Reference: {reference[:80]}...")
        print(f"  Prediction: {prediction[:80]}...")
        print(f"  Length - Ref: {len(reference)}, Pred: {len(prediction)}")
        print()

# Calculate metrics
evaluator = HTREvaluator({'case_sensitive': True, 'normalize_whitespace': True})
test_metrics = evaluator.compute_metrics(predictions, references)

print("📈 FINAL EVALUATION RESULTS:")
print("=" * 40)
for metric, value in test_metrics.items():
    if metric in ['cer', 'wer', 'exact_match', 'similarity_ratio', 'avg_edit_distance']:
        print(f"  {metric.upper()}: {value:.4f}")

# Compare với baseline (random)
print(f"\n🎯 PERFORMANCE ANALYSIS:")
print(f"  Training samples: 25")
print(f"  Training epochs: 20")
print(f"  Training time: ~19 minutes")
print(f"  Final training loss: 5.699")
print(f"  Validation loss: 6.559")

if test_metrics['cer'] < 0.9:
    print(f"  🎉 CER < 90% - Model học được pattern cơ bản!")
else:
    print(f"  ⚠️ CER cao - Cần thêm data hoặc tuning")

if test_metrics['similarity_ratio'] > 0.2:
    print(f"  ✅ Similarity > 20% - Model có tiềm năng")
else:
    print(f"  ❌ Similarity thấp - Cần cải thiện")

📊 Comprehensive Model Evaluation
✅ Trained model loaded for evaluation
🔍 Evaluating on 4 test samples...
Sample 1:
  Reference: 1818 (de Mme la Princesse de Salm) Lettre A Mr Raboteau Dyck, ce 6 Décembre, 181...
  Prediction: 18 ( M. A. M la.,.. de. la,, de,é.'.é,', le. à. et. le, à, et, d. v. d, v, je. j...
  Length - Ref: 3580, Pred: 383

Sample 2:
  Reference: Lettres ces gens là ne seront jamais heureux, ils ne méritent pas de l'être, ils...
  Prediction: Lre Mart Madame Madame,,..','., de. de'' de,é.é'é de deéé, et. et, v, à, le, d, ...
  Length - Ref: 2619, Pred: 337

Sample 3:
  Reference: Lettre de Martini Madame. Je viens de recevoir votre petite lettre mais trop tar...
  Prediction: Lre Mart Madame Mart..,.'. de',' de,, de.é'é deé,é. et, et. v. le.re.i. d. à. l....
  Length - Ref: 1315, Pred: 436

Sample 4:
  Reference: Lettre de Martini Madame Je vous ai vu passer au haut de ma rue avant hier à 2 h...
  Prediction: Lre Martiniett Madame Madame..'.,' de. de'',, de,.é'é,é de d